In [2]:
import pandas as pd
import numpy as np
import glob
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR, SVC
from dateutil import parser

In [3]:
scores_df = pd.read_csv("LW_Manual_scores_for_ICC_2020-05-20.csv")

#hit scores
hit_df = scores_df[["subject","date","crossing number","run","limb","human_hit_1","human_hit_2","human_hit_3"]]
#take the mode. If no mode, take the mean and round up
hit_df.loc[:,"human_hit"]=hit_df[["human_hit_1","human_hit_2","human_hit_3"]].mode(axis=1).mean(axis=1).round(0)

#score extraction
dom_f_hit = hit_df.loc[hit_df["limb"]=="Dominant Front"]


/home/isabelle/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/isabelle/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [4]:
rat_folder = glob.glob("*/dlc_output_resnet50/*.h5")
rung_folder = glob.glob("*/dlc_rung_r50/*.h5")

In [7]:
X = []
y = []
lengths = []
for file in rat_folder:
    #rat tracking file
    #name of the rung file
    rung_name_list = file.split("/")[-1].split("_")[0:8]+["LadderWalkMar12shuffle1","450000.h5"]
    rung_file = '_'.join(rung_name_list)
    #open the file
    rat_df = pd.read_hdf(file)["DLC_resnet50_LadderWalkFeb13shuffle1_450000"]
    #properties of the file
    subject = file.split("/")[0]
    date_raw = rung_name_list[1]
    date = parser.parse(date_raw).date().strftime("%Y-%m-%d")
    run = rung_name_list[2]
    crossing = [int(s) for s in rung_name_list[3] if s.isdigit()][0]
    #
    dom_f_hit_score = dom_f_hit[(dom_f_hit["subject"]==subject) & (dom_f_hit["date"]==date) & (dom_f_hit["run"]==run)].reset_index()
    if len(dom_f_hit_score) == 0:
        print(subject + date + run + "not found")
        continue
    
    
    #open corresponding rung file
    #rung_df = pd.read_hdf(subject+"/dlc_rung_r50/"+rung_file)["DLC_resnet50_LadderWalkMar12shuffle1_450000"]

    #join the rat and rung dataframes
    #merge_df = rat_df.join(rung_df)
    #dom_f_cols = ['right elbow','right wrist','right fingers']
    dom_f_cols = rat_df.columns.tolist()


    #new dataframe with columns removed
    #dom_f = merge_df[dom_f_cols_up]
    dom_f_temp = rat_df[dom_f_cols]
    
    #remove where median likelihood is low
    dom_f_low_like = dom_f_temp.columns[-dom_f_temp[dom_f_temp.columns.get_level_values(0).unique()].median().ge(0.2)].get_level_values(0).tolist()
    dom_f_cols_up = [x for x in dom_f_cols if x not in dom_f_low_like ]

    dom_f = rat_df[dom_f_cols_up]
    if dom_f.shape[1] == 0:
        continue
    y.append(dom_f_hit_score["human_hit"][0])
    #if len(dom_f_low_like) >0:
        #print(dom_f_low_like)
    dom_f = dom_f.drop('likelihood', axis=1, level=1)

    #scale data
    ####Uses the values function of pandas - converts any dataframe to an array
    data_for_scaling = dom_f.values
    #### the scaling object
    scaler = MinMaxScaler()
    #We will use fit_transform here - we want to actually scale this data, not use the scaler on a different dataset
    scaled_data = scaler.fit_transform(data_for_scaling)
    #PCA
    #data_reducer = PCA(n_components=5)
    #pca_data = data_reducer.fit_transform(scaled_data)
    ''' This loop gives us the top feature of each component using the argmax function '''
    #top_features = [np.abs(data_reducer.components_[i]).argmax() for i in range(data_reducer.components_.shape[0])]


    ''' Now, we go back to our original feature names (columns of our dataset), and get the names of the 4 key features '''
    #top_feature_names = [list(dom_f.columns)[top_features[i]] for i in range(data_reducer.components_.shape[0])]

    ''' printing the names '''
    #for n,name in enumerate(top_feature_names):
        #print("#" + str(n) + ": " + str(name))

    #print(data_reducer.components_)
    #print(data_reducer.explained_variance_ratio_)
    
    
    if len(scaled_data) < 356:
        newlength = (356-len(scaled_data))
        zero = np.zeros((newlength,6))
        scaled_temp = pd.DataFrame(scaled_data).append(pd.DataFrame(zero),ignore_index=True)
        scaled_temp = scaled_temp.fillna(0)
        scaled_data2 = scaled_temp.values
    else:
        scaled_data2 = scaled_data
    
    video_data = scaled_data2.flatten()
    lengths.append([len(video_data)])
    X.append(video_data)
    
    

MC702019-03-14L4not found
MC702019-04-09L4not found


In [17]:
with np.printoptions(threshold=np.inf):
    print(video_data)

[0.00000000e+00 9.73311433e-01 0.00000000e+00 8.33956051e-01
 8.67803618e-02 7.13453682e-01 0.00000000e+00 7.97765800e-01
 0.00000000e+00 8.90292304e-01 0.00000000e+00 8.68947826e-01
 1.83724697e-02 6.86855801e-01 9.03241534e-04 7.44578193e-01
 0.00000000e+00 7.86011143e-01 0.00000000e+00 7.56328356e-01
 1.00238900e-02 6.05969602e-01 0.00000000e+00 3.11713056e-01
 0.00000000e+00 6.06843664e-01 0.00000000e+00 5.53027155e-01
 6.74425731e-03 9.61861518e-01 0.00000000e+00 4.56881558e-01
 6.82713586e-03 9.70649740e-01 3.63625512e-03 8.54344422e-01
 9.49497796e-02 6.41287312e-01 4.77698834e-03 7.82570703e-01
 1.23667313e-02 8.71870658e-01 1.02033914e-01 4.87704845e-01
 9.27843111e-02 5.88185089e-01 3.39308621e-03 7.18605497e-01
 7.23063399e-03 7.44964693e-01 5.51876469e-03 7.56852934e-01
 7.88213720e-03 6.64316161e-01 7.07557242e-02 9.03275660e-01
 8.22433392e-02 6.88769193e-01 5.98526660e-02 9.56799372e-01
 0.00000000e+00 9.58296034e-01 1.14135663e-02 9.23592147e-01
 1.17461181e-02 9.274506

In [15]:
pd.DataFrame(scaled_data)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.000000,0.973311,0.000000,0.833956,0.086780,0.713454,0.000000,0.797766,0.000000,0.890292,...,0.000000,0.311713,0.000000,0.606844,0.000000,0.553027,0.006744,0.961862,0.000000,0.456882
1,0.006827,0.970650,0.003636,0.854344,0.094950,0.641287,0.004777,0.782571,0.012367,0.871871,...,0.070756,0.903276,0.082243,0.688769,0.059853,0.956799,0.000000,0.958296,0.011414,0.923592
2,0.011746,0.927451,0.010297,0.737634,0.205884,0.604660,0.007844,0.739091,0.019332,0.763784,...,0.077581,1.000000,0.098713,0.779074,0.065182,1.000000,0.017302,0.957858,0.012623,0.931693
3,0.017885,0.769464,0.012507,0.595420,0.209411,0.547273,0.011042,0.748133,0.029659,0.764885,...,0.079616,0.968234,0.099168,0.765252,0.063089,0.964954,0.028954,0.944272,0.012392,0.912395
4,0.030721,0.776056,0.023515,0.619770,0.000000,0.628869,0.018153,0.760121,0.044432,0.791838,...,0.080078,0.929299,0.100067,0.839194,0.065701,0.938376,0.031361,0.914741,0.017163,0.874434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,0.961059,0.209279,0.958483,0.256953,0.964951,0.448752,0.948725,0.390420,0.983745,0.280261,...,0.903221,0.320474,0.891305,0.321718,0.878609,0.407303,0.878861,0.255351,0.929315,0.323997
105,0.963540,0.363468,0.962218,0.425395,0.969597,0.509801,0.960769,0.583514,0.983301,0.359032,...,0.904165,0.347030,0.890643,0.345129,0.877466,0.386963,0.881930,0.232225,0.934299,0.324882
106,0.976105,0.268275,0.973533,0.391644,0.978336,0.500519,0.971842,0.594011,0.986035,0.299924,...,0.906781,0.204977,0.892596,0.343407,0.803325,0.000000,0.885046,0.231840,0.941602,0.340756
107,0.988427,0.295977,0.985605,0.407956,0.989502,0.513931,0.984821,0.602861,0.989662,0.285663,...,0.963338,0.268533,0.907434,0.104033,0.890094,0.338513,0.903609,0.395814,0.953939,0.369062


In [14]:
X = np.array(X)
y = np.array(y)

In [15]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [16]:
regressor = SVR(kernel="rbf")
regressor.fit(train_features, train_labels)
y_pred = regressor.predict(X)

In [17]:
# Use the forest's predict method on the test data
predictions = regressor.predict(test_features)# Calculate the absolute errors
errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.66 degrees.


In [18]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 81.4 %.


In [ ]:
y-y_pred.round(0)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y,y_pred.round(0),squared=True)

In [ ]:
regressor = SVC(kernel="rbf")
regressor.fit(X,y)
y_pred = regressor.predict(X)

In [ ]:
y-y_pred

In [ ]:
mean_squared_error(y,y_pred.round(0),squared=True)

In [ ]:
set(y)